In [1]:
import matplotlib.pyplot as plt
import fitbit

# gather_keys_oauth2.py file needs to be in the same directory. 
# also needs to install cherrypy: https://pypi.org/project/CherryPy/
# pip install CherryPy
import gather_keys_oauth2 as Oauth2
import pandas as pd 
import datetime


CLIENT_ID='22BYG2'
CLIENT_SECRET='12738caef1144c4f1feacda3dfcc4c5a'

In [2]:

server=Oauth2.OAuth2Server(CLIENT_ID, CLIENT_SECRET)
server.browser_authorize()
ACCESS_TOKEN=str(server.fitbit.client.session.token['access_token'])
REFRESH_TOKEN=str(server.fitbit.client.session.token['refresh_token'])
auth2_client=fitbit.Fitbit(CLIENT_ID,CLIENT_SECRET,oauth2=True,access_token=ACCESS_TOKEN,refresh_token=REFRESH_TOKEN)

[15/Jan/2021:15:31:32] ENGINE Listening for SIGTERM.
[15/Jan/2021:15:31:32] ENGINE Bus STARTING
[15/Jan/2021:15:31:32] ENGINE Set handler for console events.
CherryPy Checker:
The Application mounted at '' has an empty config.

[15/Jan/2021:15:31:32] ENGINE Started monitor thread 'Autoreloader'.
[15/Jan/2021:15:31:32] ENGINE Serving on http://127.0.0.1:8080
[15/Jan/2021:15:31:32] ENGINE Bus STARTED


127.0.0.1 - - [15/Jan/2021:15:31:34] "GET /?code=6281b578da51e70e4096c22da7e0550ed675cff2&state=OdiwDihkn5UC2m5s7h1C3AAtOk3lxq HTTP/1.1" 200 122 "" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36"


[15/Jan/2021:15:31:35] ENGINE Bus STOPPING
[15/Jan/2021:15:31:35] ENGINE HTTP Server cherrypy._cpwsgi_server.CPWSGIServer(('127.0.0.1', 8080)) shut down
[15/Jan/2021:15:31:35] ENGINE Stopped thread 'Autoreloader'.
[15/Jan/2021:15:31:35] ENGINE Removed handler for console events.
[15/Jan/2021:15:31:35] ENGINE Bus STOPPED
[15/Jan/2021:15:31:35] ENGINE Bus EXITING
[15/Jan/2021:15:31:35] ENGINE Bus EXITED
[15/Jan/2021:15:31:35] ENGINE Waiting for child threads to terminate...


In [3]:
oneDate = pd.datetime(year = 2021, month = 1, day = 11)

<ipython-input-3-9a5f2ec19ca3>:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  oneDate = pd.datetime(year = 2021, month = 1, day = 11)


In [4]:
help(auth2_client.intraday_time_series)

Help on method intraday_time_series in module fitbit.api:

intraday_time_series(resource, base_date='today', detail_level='1min', start_time=None, end_time=None) method of fitbit.api.Fitbit instance
    The intraday time series extends the functionality of the regular time series, but returning data at a
    more granular level for a single day, defaulting to 1 minute intervals. To access this feature, one must
    fill out the Private Support form here (see https://dev.fitbit.com/docs/help/).
    For details on the resources available and more information on how to get access, see:
    
    https://dev.fitbit.com/docs/activity/#get-activity-intraday-time-series



In [5]:
oneDayData = auth2_client.intraday_time_series('activities/heart',
                                               base_date=oneDate,
                                               detail_level='1sec')

In [6]:
oneDayData

{'activities-heart': [{'dateTime': '2021-01-11',
   'value': {'customHeartRateZones': [{'caloriesOut': 0,
      'max': 30,
      'min': 30,
      'minutes': 0,
      'name': 'Below'},
     {'caloriesOut': 0,
      'max': 31,
      'min': 30,
      'minutes': 0,
      'name': 'Custom Zone'},
     {'caloriesOut': 1858.9575,
      'max': 220,
      'min': 31,
      'minutes': 792,
      'name': 'Above'}],
    'heartRateZones': [{'caloriesOut': 2464.9425,
      'max': 112,
      'min': 30,
      'minutes': 1425,
      'name': 'Out of Range'},
     {'caloriesOut': 87.2235,
      'max': 140,
      'min': 112,
      'minutes': 15,
      'name': 'Fat Burn'},
     {'caloriesOut': 0,
      'max': 176,
      'min': 140,
      'minutes': 0,
      'name': 'Cardio'},
     {'caloriesOut': 0, 'max': 220, 'min': 176, 'minutes': 0, 'name': 'Peak'}],
    'restingHeartRate': 55}}],
 'activities-heart-intraday': {'dataset': [{'time': '00:00:05', 'value': 97},
   {'time': '00:00:10', 'value': 97},
   {'time

In [7]:

df = pd.DataFrame(oneDayData['activities-heart-intraday']['dataset'])

In [8]:
df.head()

,time,value
0,00:00:05,97
1,00:00:10,97
2,00:00:15,100
3,00:00:20,102
4,00:00:25,104


In [9]:
filename = oneDayData['activities-heart'][0]['dateTime'] +'_intradata'

# Export file to csv
df.to_csv(filename + '.csv', index = False)
df.to_excel(filename + '.xlsx', index = False)

In [10]:
startTime = pd.datetime(year = 2020, month = 12, day = 21)
endTime = pd.datetime.today().date() - datetime.timedelta(days=1)

<ipython-input-10-4865c1a524c2>:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  startTime = pd.datetime(year = 2020, month = 12, day = 21)
<ipython-input-10-4865c1a524c2>:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  endTime = pd.datetime.today().date() - datetime.timedelta(days=1)


In [11]:
date_list = []
df_list = []
allDates = pd.date_range(start=startTime, end = endTime)

for oneDate in allDates:
    
    oneDate = oneDate.date().strftime("%Y-%m-%d")
    
    oneDayData = auth2_client.intraday_time_series('activities/heart', base_date=oneDate, detail_level='1sec')

    df = pd.DataFrame(oneDayData['activities-heart-intraday']['dataset'])
    
    date_list.append(oneDate)
    
    df_list.append(df)
    
final_df_list = []

for date, df in zip(date_list, df_list):

    if len(df) == 0:
        continue
    
    df.loc[:, 'date'] = pd.to_datetime(date)
    
    final_df_list.append(df)

final_df = pd.concat(final_df_list, axis = 0)

In [12]:
filename = 'all_intradata_1'
final_df.to_csv(filename + '.csv', index = False)

In [13]:
oneDayData2 = auth2_client.intraday_time_series('activities/distance',
                                               base_date=oneDate,
                                               detail_level='1min')

In [14]:
oneDayData2

{'activities-distance': [{'dateTime': '2021-01-10',
   'value': '2.35339989371656'}],
 'activities-distance-intraday': {'dataset': [{'time': '00:00:00', 'value': 0},
   {'time': '00:01:00', 'value': 0},
   {'time': '00:02:00', 'value': 0},
   {'time': '00:03:00', 'value': 0},
   {'time': '00:04:00', 'value': 0},
   {'time': '00:05:00', 'value': 0},
   {'time': '00:06:00', 'value': 0},
   {'time': '00:07:00', 'value': 0},
   {'time': '00:08:00', 'value': 0},
   {'time': '00:09:00', 'value': 0},
   {'time': '00:10:00', 'value': 0},
   {'time': '00:11:00', 'value': 0},
   {'time': '00:12:00', 'value': 0},
   {'time': '00:13:00', 'value': 0},
   {'time': '00:14:00', 'value': 0},
   {'time': '00:15:00', 'value': 0},
   {'time': '00:16:00', 'value': 0},
   {'time': '00:17:00', 'value': 0},
   {'time': '00:18:00', 'value': 0},
   {'time': '00:19:00', 'value': 0},
   {'time': '00:20:00', 'value': 0},
   {'time': '00:21:00', 'value': 0},
   {'time': '00:22:00', 'value': 0},
   {'time': '00:23:0

In [15]:
d = pd.DataFrame(oneDayData2['activities-distance-intraday']['dataset'])

In [16]:
d.head()

,time,value
0,00:00:00,0.0
1,00:01:00,0.0
2,00:02:00,0.0
3,00:03:00,0.0
4,00:04:00,0.0


In [17]:
dayDatacalories= auth2_client.intraday_time_series('activities/calories',base_date=oneDate,detail_level='1min')

In [18]:
dayDatacalories

{'activities-calories': [{'dateTime': '2021-01-10', 'value': '2364'}],
 'activities-calories-intraday': {'dataset': [{'level': 1,
    'mets': 35,
    'time': '00:00:00',
    'value': 3.7274999618530273},
   {'level': 1, 'mets': 34, 'time': '00:01:00', 'value': 3.621000051498413},
   {'level': 1, 'mets': 34, 'time': '00:02:00', 'value': 3.621000051498413},
   {'level': 1, 'mets': 32, 'time': '00:03:00', 'value': 3.4079999923706055},
   {'level': 1, 'mets': 33, 'time': '00:04:00', 'value': 3.5144999027252197},
   {'level': 1, 'mets': 35, 'time': '00:05:00', 'value': 3.7274999618530273},
   {'level': 1, 'mets': 32, 'time': '00:06:00', 'value': 3.4079999923706055},
   {'level': 1, 'mets': 34, 'time': '00:07:00', 'value': 3.621000051498413},
   {'level': 1, 'mets': 33, 'time': '00:08:00', 'value': 3.5144999027252197},
   {'level': 1, 'mets': 33, 'time': '00:09:00', 'value': 3.5144999027252197},
   {'level': 1, 'mets': 33, 'time': '00:10:00', 'value': 3.5144999027252197},
   {'level': 0, 'me

In [19]:
d_calories = pd.DataFrame(dayDatacalories['activities-calories-intraday']['dataset'])

In [20]:
d_calories

,level,mets,time,value
0,1,35,00:00:00,3.7275
1,1,34,00:01:00,3.6210
2,1,34,00:02:00,3.6210
3,1,32,00:03:00,3.4080
4,1,33,00:04:00,3.5145
...,...,...,...,...
1435,3,60,23:55:00,6.3900
1436,2,46,23:56:00,4.8990
1437,2,46,23:57:00,4.8990
1438,1,42,23:58:00,4.4730


In [83]:
act = auth2_client.intraday_time_series('sleep/stages',
                                               base_date=oneDate,
                                               detail_level='1min')

HTTPBadRequest: Invalid time series resource path: /sleep/stages

In [25]:
oneDayData2 = auth2_client.intraday_time_series('body/weight',
                                               base_date=oneDate,
                                               detail_level='1min')

In [26]:
oneDayData2

{'body-weight': [{'dateTime': '2021-01-14', 'value': '132.27'}]}